# Extract problematic frames from a label video

Let's say you have trained a model, analyzed and labeled video. By watching the label video, you might notice that you model is making some mistakes. 

You can fix this by extracing the problematic frames, manually label the body parts and adding these images to your dataset, and training your model further. 

This notebook helps with extracting new images for which you model is not performing well.

In [1]:
%run setup_project.py
%matplotlib widget
from unetTracker.dataset import UNetDataset
from unetTracker.unetGUI import LabelFromImagesGUI, SelectImagesFromLabeledVideoGUI

Project directory: /home/kevin/Documents/trackingProjects/finger_tracker
Getting configuration from config file. Values from config file will be used.
Loading /home/kevin/Documents/trackingProjects/finger_tracker/config.yalm
{'augmentation_HorizontalFlipProb': 0.0, 'augmentation_RandomBrightnessContrastProb': 0.2, 'augmentation_RandomSizedCropProb': 1.0, 'augmentation_RotateProb': 0.3, 'image_extension': '.png', 'image_size': [270, 480], 'labeling_ImageEnlargeFactor': 2.0, 'name': 'finger_tracker', 'normalization_values': {'means': [0.40835028886795044, 0.4549056589603424, 0.51627117395401], 'stds': [0.23996737599372864, 0.251758873462677, 0.26929107308387756]}, 'object_colors': [(0.0, 0.0, 255.0), (255.0, 0.0, 0.0), (255.0, 255.0, 0.0), (240.0, 255.0, 255.0)], 'objects': ['f1', 'f2', 'f3', 'f4'], 'target_radius': 6, 'unet_features': [64, 128, 256, 512]}


In [2]:
dataset = UNetDataset(image_dir=project.image_dir, mask_dir=project.mask_dir, coordinate_dir=project.coordinate_dir,
                      image_extension=project.image_extension)
print("Number of images in the dataset:",len(dataset))

Number of images in the dataset: 184


In [4]:
video_fn = "/home/kevin/Documents/trackingProjects/finger_tracker/videos/human_hand.mp4"
label_fn = (os.path.splitext(video_fn)[0]) + "_label.avi"
print(video_fn, label_fn)

/home/kevin/Documents/trackingProjects/finger_tracker/videos/human_hand.mp4 /home/kevin/Documents/trackingProjects/finger_tracker/videos/human_hand_label.avi


In [6]:
for fn in [video_fn,label_fn]:
    if os.path.exists(fn):
        print("Video file {} exist".format(fn))
    else:
        raise IOError("Video file {} does not exist".format(fn))

Video file /home/kevin/Documents/trackingProjects/finger_tracker/videos/human_hand.mp4 exist
Video file /home/kevin/Documents/trackingProjects/finger_tracker/videos/human_hand_label.avi exist


In [7]:
myGui = SelectImagesFromLabeledVideoGUI(label_fn, 
                                project)
display(myGui)

SelectImagesFromLabeledVideoGUI(children=(HBox(children=(Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x0…

In [8]:
myList = myGui.frame_list
print(len(myList))
myList[:10]

16


[462, 546, 950, 1029, 1139, 1237, 1321, 1542, 1594, 1710]

In [9]:
extracted_frame_dir = project.project_dir+"/extracted_frames/" # directory where the extracted images will be saved
frame_info_file = extracted_frame_dir+"frame_info.csv" # file where the source of the image is stored



dataset.extract_frames_from_video(video_fn = video_fn, 
                                  frame_dir = extracted_frame_dir,
                                  image_size = project.image_size,
                                  selected_frames = myList,
                                  frame_info_file = frame_info_file)

video length: 5795, image size: 270h 480w
Extracting frames: [462, 546, 950, 1029, 1139, 1237, 1321, 1542, 1594, 1710, 1821, 1944, 2308, 2311, 2590, 2796] to /home/kevin/Documents/trackingProjects/finger_tracker/extracted_frames/
Saving image info to /home/kevin/Documents/trackingProjects/finger_tracker/extracted_frames/frame_info.csv


In [10]:
LabelFromImagesGUI(image_dir=extracted_frame_dir,
                   frame_info_file=frame_info_file,
                   project=project,dataset=dataset)

frame.shape: (270, 480, 3)
b45c544e-893a-11ee-8baa-cb80e822b7fc.png


LabelFromImagesGUI(children=(HTML(value='Event info'), HBox(children=(Label(value='Objects:'), RadioButtons(la…

In [11]:
len(dataset)

200

Now that you have added problematic frames to your dataset, you can use them to further train your network you will have to go through Notebooks 04_ to 06_ to retrain your network. 